In [1]:
#import the libraries as needed
from selenium import webdriver
from selenium import webdriver   # for webdriver
from selenium.webdriver.support.ui import WebDriverWait  # for implicit and explict waits
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.chrome.service import Service
import numpy as np
import json
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm # to create loadbard in for loop
import os.path
import pickle

In [2]:
# need to precise the option for date, nbr of metada for call transcript in the url paramater 
# https://en.wikipedia.org/wiki/Query_string
nbr_earning_call_metada = 200 # I think 50 is max but can easyily be adapted, with for loop or something like that
date_of_request= "2022-04-28"
url = "https://seekingalpha.com/api/v3/articles"+"?cacheBuster="\
      +str(date_of_request)\
      +"&filter[category]=earnings%3A%3Aearnings-call-transcripts&filter[since]=0"\
      +"&filter[until]=0&include=author%2CprimaryTickers%2CsecondaryTickers&isMounting=true&page[size]="\
      +str(nbr_earning_call_metada)\
      +"&page[number]=1"
url

'https://seekingalpha.com/api/v3/articles?cacheBuster=2022-04-28&filter[category]=earnings%3A%3Aearnings-call-transcripts&filter[since]=0&filter[until]=0&include=author%2CprimaryTickers%2CsecondaryTickers&isMounting=true&page[size]=200&page[number]=1'

In [3]:
# the chrome driver depend on which version of chrome you are running specify the good one
executable_path = "./chromedriver_linux"
option = webdriver.ChromeOptions()
driver = webdriver.Chrome(executable_path= executable_path,options=option)
driver.get(url)
site_html = driver.page_source

/home/thomas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  after removing the cwd from sys.path.


In [4]:
#a quick loock of the result to check if we get results are get captcha/bot error 
site_html[0:300]

'<html><head></head><body><pre style="word-wrap: break-word; white-space: pre-wrap;">{"data":[{"id":"4505090","type":"transcript","attributes":{"publishOn":"2022-04-29T10:24:08-04:00","isLockedPro":false,"commentCount":0,"gettyImageUrl":null,"themes":{},"title":"Shenandoah Telecommunications Company '

In [5]:
api_json_response = BeautifulSoup(site_html)
api_data_response = json.loads(api_json_response.get_text())

## API data response

<pre>
dict( 
    data : list(one element is one earning call metada)
    included : "don't care for now"
    meta : "don't care for now"
        )
<pre>
One earning call metadata is show below 

In [6]:
api_data_response["data"][0]

{'id': '4505090',
 'type': 'transcript',
 'attributes': {'publishOn': '2022-04-29T10:24:08-04:00',
  'isLockedPro': False,
  'commentCount': 0,
  'gettyImageUrl': None,
  'themes': {},
  'title': 'Shenandoah Telecommunications Company (SHEN) CEO Christopher French on Q1 2022 Results - Earnings Call Transcript'},
 'relationships': {'author': {'data': {'id': '44211', 'type': 'author'}},
  'sentiments': {'data': []},
  'primaryTickers': {'data': [{'id': '10060', 'type': 'tag'}]},
  'secondaryTickers': {'data': []},
  'otherTags': {'data': [{'id': '96991', 'type': 'tag'},
    {'id': '17994', 'type': 'tag'},
    {'id': '49', 'type': 'tag'},
    {'id': '586376', 'type': 'tag'},
    {'id': '326', 'type': 'tag'},
    {'id': '18024', 'type': 'tag'}]}},
 'links': {'self': '/article/4505090-shenandoah-telecommunications-company-shen-ceo-christopher-french-on-q1-2022-results-earnings'}}

In [7]:

id_and_path_to_earning_calls = [ (earning_call_metada["id"] ,earning_call_metada["links"]["self"]) 
                     for earning_call_metada in api_data_response["data"]]
# ""check url
id_and_path_to_earning_calls[0:2]

[('4505090',
  '/article/4505090-shenandoah-telecommunications-company-shen-ceo-christopher-french-on-q1-2022-results-earnings'),
 ('4505087',
  '/article/4505087-noranda-income-fund-nndif-ceo-paul-einarson-on-q1-2022-results-earnings-call-transcript')]

## download all not yet download html from url

In [8]:
domain_name = 'https://seekingalpha.com'
need_to_download = []
for id_earning_call, path_to_earning_call in tqdm(id_and_path_to_earning_calls):
    if not os.path.isfile("data/article/"+ id_earning_call +".html"):
        # not in local directory
        need_to_download.append((id_earning_call, path_to_earning_call))
print(f"Need to download {len(need_to_download)} html file(s)")
for id_earning_call, path_to_earning_call in tqdm(need_to_download):
    with open("data/article/"+ id_earning_call+".html", "xb") as file:
            url = domain_name + path_to_earning_call
            html_transcript = requests.get(url).text
            pickle.dump(html_transcript, file)
    

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 16274.65it/s]


Need to download 0 html file(s)


0it [00:00, ?it/s]


## Check last file got downloaded, i.e. captcha/bot error 

In [9]:
BeautifulSoup(html_transcript).get_text()[:2000]

NameError: name 'html_transcript' is not defined